In [ ]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

# Importing data for API call
from config import app_token
from config import username
from config import password

In [ ]:
# Getting the data from API NY Open Data for the first database: 
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)
Recycling_Div_Captures

In [ ]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"
# Read the data into df
cws_df = pd.read_csv(data_path)
#Preview data
cws_df.head()

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [ ]:
df_list = []
for x in range(1,41):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-visualization')
    df_list.append(pd.read_html(str(table))[0]) 
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[2]/span[1]/button[2]').click()

In [ ]:
df_combined = pd.concat(df_list)

In [ ]:
df_combined.reset_index(drop=True, inplace=True)

In [ ]:
cwm_df = df_combined
cwm_df.head()

In [ ]:
# Analizing material groups
unique_mat_subsort = cws_df["Material Group"].unique()
print(f'Material groups (subsort): {unique_mat_subsort}')
unique_mat_mainsort = cwm_df["Material Group"].unique()
print(f'Material groups (mainsort): {unique_mat_mainsort}')

In [ ]:
unique_loc_subsort = cws_df["Location"].unique()
print(f"Locations register in the Subsort dataset: {unique_loc_subsort}")
print("----------------------------------------------------------------")
unique_loc_mainsort = cwm_df["Location"].unique()
print(f"Locations register in the Mainsort dataset: {unique_loc_mainsort}")
print("----------------------------------------------------------------")
uniqie_loc_general = Recycling_Div_Captures["_zone"].unique()
print(f"Locations register in the general dataset: {uniqie_loc_general}")

In [ ]:
# Format change in the general dataset
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn North', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn South', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens East', r'Queens')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens West', r'Queens')

In [ ]:
# First filter: Recycling_Div_Captures data should be filter just for 2017 year
Recycling_Div_Captures["fiscal_year"] = Recycling_Div_Captures["fiscal_year"].apply(pd.to_numeric, errors='coerce')
Recycling_Div_Captures = Recycling_Div_Captures.loc[Recycling_Div_Captures["fiscal_year"]==2017,:]

In [ ]:
# Rename columns and convert to_numeric 
Recycling_Div_Captures = Recycling_Div_Captures.rename(columns={
    "diversion_rate_total_total_recycling_total_waste_": "Total recycling rate",
    "capture_rate_paper_total_paper_max_paper_": "Paper rate",
    "capture_rate_mgp_total_mgp_max_mgp_": "MGP rate"
})
cols = ["Total recycling rate", "Paper rate", "MGP rate"]
Recycling_Div_Captures[cols] = Recycling_Div_Captures[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Subsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cws_df[cols] = cws_df[cols] *100
cws_df.head()

In [ ]:
# Mainsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cwm_df["Aggregate Percent"]= cwm_df["Aggregate Percent"].str.replace(r'%', r'')
cwm_df["Refuse Percent"]= cwm_df["Refuse Percent"].str.replace(r'%', r'')
cwm_df["MGP Percent"]= cwm_df["MGP Percent"].str.replace(r'%', r'')
cwm_df["Paper Percent"]= cwm_df["Paper Percent"].str.replace(r'%', r'')
cwm_df["Organic Percent"]= cwm_df["Organic Percent"].str.replace(r'%', r'')

cwm_df[cols]= cwm_df[cols].apply(pd.to_numeric, errors='coerce')
print(cwm_df.dtypes)
cwm_df.head()

In [ ]:
# Paper collection
# Materials paper types register in subsort
paper_material_sub = cws_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_sub = paper_material_sub["Material"].unique()
set_sub = set(paper_material_sub)

# Organic collection
# Materials Organic types register in subsort
org_material_sub = cws_df.loc[cws_df["Material Group"]=="Organic",["Material"]]
org_material_sub = org_material_sub["Material"].unique()
set_sub_o = set(org_material_sub)

# E-waste collection
# Materials ewaste types register in subsort
ew_material_sub = cws_df.loc[cws_df["Material Group"]=="E-Waste",["Material"]]
ew_material_sub = ew_material_sub["Material"].unique()
set_sub_e = set(ew_material_sub)

In [ ]:
# Materials paper types register in mainsort
paper_material_main = cwm_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_main = paper_material_main["Material"].unique()
set_main = set(paper_material_main)

# Materials paper types register in mainsort
org_material_main = cwm_df.loc[cws_df["Material Group"]=="Organic",["Material"]]
org_material_main = org_material_main["Material"].unique()
set_main_o = set(org_material_main)

# Materials paper types register in mainsort
ew_material_main = cwm_df.loc[cws_df["Material Group"]=="E-Waste",["Material"]]
ew_material_main = ew_material_main["Material"].unique()
set_main_e = set(ew_material_main)

In [ ]:
# Mergin material without duplication 
no_duplicates = list(set_main-set_sub)
paper_material_sub = list(set_sub)
paper_material_final = paper_material_sub + no_duplicates
paper_material_final

no_duplicates_o = list(set_main_o-set_sub_o)
org_material_sub = list(set_sub_o)
org_material_final = org_material_sub + no_duplicates_o
print(org_material_final)

no_duplicates_e = list(set_main_e-set_sub_e)
ew_material_sub = list(set_sub_e)
ew_material_final = ew_material_sub + no_duplicates_e
print(ew_material_final)

In [ ]:
# Filter just papper material group subsort
paper_sub_df = cws_df.loc[cws_df["Material Group"]=="Paper",:]
# Filter just papper material group mainsort
paper_main_df = cwm_df.loc[cws_df["Material Group"]=="Paper",:]


# Filter just organic material group subsort
org_sub_df = cws_df.loc[cws_df["Material Group"]=="Organic",:]
# Filter just organic material group mainsort
org_main_df = cwm_df.loc[cws_df["Material Group"]=="Organic",:]

# Filter just e-waste material group subsort
ew_sub_df = cws_df.loc[cws_df["Material Group"]=="E-Waste",:]
# Filter just e-waste material group mainsort
ew_main_df = cwm_df.loc[cws_df["Material Group"]=="E-Waste",:]

# Group and calculate the mean of aggregate and refuse percentage per location
agg_subsort = paper_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort = paper_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort = paper_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort = paper_main_df.groupby("Location")["Refuse Percent"].mean()


# Group and calculate the mean of aggregate and refuse percentage per location for organic material
agg_subsort_o = org_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort_o = org_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort_o = org_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort_o = org_main_df.groupby("Location")["Refuse Percent"].mean()


# Group and calculate the mean of aggregate and refuse percentage per location for e waste material
agg_subsort_e = ew_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort_e = ew_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort_e = ew_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort_e = ew_main_df.groupby("Location")["Refuse Percent"].mean()

# Collection paper will have Average capture rate taken from Recycling_Div_Captures DataFrame
avg_cap_rate = Recycling_Div_Captures.groupby("_zone")["Paper rate"].mean()


In [ ]:
# MGP collection
# 'Plastic' 'Glass' 'Metal'
# Types of Metal register in subsort
Metals_sub = cws_df.loc[(cws_df["Material Group"]=="Metal"),["Material"]]
Metals_sub = Metals_sub["Material"].unique()
set_sub = set(Metals_sub)
# Types of MGP materials register in mainsort
Metals_main = cwm_df.loc[(cwm_df["Material Group"]=="Metal"),["Material"]]
Metals_main = Metals_main["Material"].unique()
set_main = set(Metals_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Metals_sub = list(set_sub)
Metals_final= Metals_sub + no_duplicates
Metals_final

In [ ]:
# Types of glass register in subsort
Glass_sub = cws_df.loc[(cws_df["Material Group"]=="Glass"),["Material"]]
Glass_sub = Glass_sub["Material"].unique()
set_sub = set(Glass_sub)
# Types of MGP materials register in mainsort
Glass_main = cwm_df.loc[(cwm_df["Material Group"]=="Glass"),["Material"]]
Glass_main = Glass_main["Material"].unique()
set_main = set(Glass_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Glass_sub = list(set_sub)
Glass_final= Glass_sub + no_duplicates
Glass_final

In [ ]:
# Types of plastic register in subsort
Plas_sub = cws_df.loc[(cws_df["Material Group"]=="Plastic"),["Material"]]
Plas_sub = Plas_sub["Material"].unique()
set_sub = set(Plas_sub)
# Types of MGP materials register in mainsort
Plas_main = cwm_df.loc[(cwm_df["Material Group"]=="Plastic"),["Material"]]
Plas_main = Plas_main["Material"].unique()
set_main = set(Plas_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Plas_sub = list(set_sub)
Plastic_final= Plas_sub + no_duplicates
Plastic_final

In [ ]:
# Filter MGP materials subsort
MGP_sub_df = cws_df.loc[(cws_df["Material Group"]=="Glass")|(cws_df["Material Group"]=="Plastic")
                              | (cws_df["Material Group"]=="Metal"),:]
# Filter MGP materials mainsort
MGP_main_df = cwm_df.loc[(cwm_df["Material Group"]=="Glass")|(cwm_df["Material Group"]=="Plastic")
                              | (cwm_df["Material Group"]=="Metal"),:]
# Group and calculate the mean of aggregate and refuse percentage per location
MGP_agg_subsort = MGP_sub_df.groupby("Location")["Aggregate Percent"].mean()
MGP_refuse_subsort = MGP_sub_df.groupby("Location")["Refuse Percent"].mean()
MGP_agg_mainsort = MGP_main_df.groupby("Location")["Aggregate Percent"].mean()
MGP_refuse_mainsort = MGP_main_df.groupby("Location")["Refuse Percent"].mean()

# Collection MGP will have Average capture rate taken from Recycling_Div_Captures DataFrame
MGP_avg_cap_rate = Recycling_Div_Captures.groupby("_zone")["MGP rate"].mean()

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
materials_db = client["MaterialsDB"]
materials_coll = materials_db["materials"]

In [ ]:
# Insert paper document in the database
materials_db.materials_coll.insert_one(
    {
        'material_group': "Paper",
        'year': 2017,
        'materials_list': paper_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Bronx,
                'Refuse Percent': refuse_subsort.Bronx,
            },
            'Capture rate': avg_cap_rate.Bronx
        },
        'Manhattan':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Manhattan,
                'Refuse Percent': refuse_subsort.Manhattan,
            },
            'Mainsort':
            {
                'Aggregate Percent': agg_mainsort.Manhattan,
                'Refuse Percent': refuse_mainsort.Manhattan,
            },
            'Capture rate': avg_cap_rate.Manhattan
        },
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Queens,
                'Refuse Percent': refuse_subsort.Queens,
            },
            'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Brooklyn,
                'Refuse Percent': refuse_subsort.Brooklyn,
            },
            'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort[7],
                'Refuse Percent': refuse_subsort[7],
            },
            'Capture rate': avg_cap_rate[4]
        }

    }
)



In [ ]:
# inserting organic waste into mongo db

materials_db.materials_coll.insert_one(
    {
        'material_group': "Paper",
        'year': 2017,
        'materials_list': org_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Bronx,
                'Refuse Percent': refuse_subsort_o.Bronx,
            },
#             'Capture rate': avg_cap_rate.Bronx
        },
#         
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Queens,
                'Refuse Percent': refuse_subsort_o.Queens,
            },
#             'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Brooklyn,
                'Refuse Percent': refuse_subsort_o.Brooklyn,
            },
#             'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o[7],
                'Refuse Percent': refuse_subsort_o[7],
            },
#             'Capture rate': avg_cap_rate[4]
        }

    }
)

In [ ]:
# inserting ewatse waste into mongo db

materials_db.materials_coll.insert_one(
    {
        'material_group': "Paper",
        'year': 2017,
        'materials_list': org_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Bronx,
                'Refuse Percent': refuse_subsort_e.Bronx,
            },
#             'Capture rate': avg_cap_rate.Bronx
        },
#         
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Queens,
                'Refuse Percent': refuse_subsort_e.Queens,
            },
#             'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Brooklyn,
                'Refuse Percent': refuse_subsort_e.Brooklyn,
            },
#             'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e[7],
                'Refuse Percent': refuse_subsort_e[7],
            },
#             'Capture rate': avg_cap_rate[4]
        }

    }
)